In [37]:
#------Code for Pulsar Dataset-----------------#
#----------Feed Forward NN--------------#
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score, confusion_matrix
#--------------------------------------------------#
# Load the data
data = pd.read_csv('pulsar_data.csv')

# Assuming 'target' is the column containing the labels
X = data.drop('target_class', axis=1)
y = data['target_class']

# Handling missing values by filling with the mean
X = X.fillna(X.mean())

# Split the data into training, testing, and validation sets

X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.2, random_state=42)
X_validation, X_test, y_validation, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

# Scale the features using MinMaxScaler
scaler = MinMaxScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_validation_scaled = scaler.transform(X_validation)
X_test_scaled = scaler.transform(X_test)


# Initialize the neural network model with 2 hidden layers and 2 nodes in each hidden layer
model = Sequential()
model.add(Dense(units=2, input_dim=X_train_scaled.shape[1], activation='relu'))
model.add(Dense(units=2, activation='relu'))
model.add(Dense(units=1, activation='sigmoid'))

# Compile the model with the Adam optimizer and a custom learning rate
custom_optimizer = Adam(learning_rate=0.001)  # Adjust the learning rate as needed
model.compile(optimizer=custom_optimizer, loss='binary_crossentropy', metrics=['accuracy'])

# Train the model and get the training history
history = model.fit(X_train_scaled, y_train, validation_data=(X_validation_scaled, y_validation), epochs=50, batch_size=32, verbose=0)

# Plot epoch vs accuracy and loss
plt.figure(figsize=(12, 5))

# Plot Accuracy
#plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Training Accuracy')
#plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.xlabel('Iterations')
plt.ylabel('Accuracy')
#plt.title('Epoch vs Accuracy')
plt.legend()
plt.savefig('FFNN_accuracy_plot.pdf', bbox_inches='tight')
plt.close()

# Plot Loss
#plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Training Loss')
#plt.plot(history.history['val_loss'], label='Validation Loss')
plt.xlabel('Iterations')
plt.ylabel('Loss')
#plt.title('Epoch vs Loss')
plt.legend()
plt.savefig('FFNN_loss_plot.pdf', bbox_inches='tight')
plt.close()  # Close the plot

plt.tight_layout()
plt.show()

# predictions on test data
predictions_test = (model.predict(X_test_scaled) > 0.5).astype(int)

# Model evaluation on the test set
accuracy_test = accuracy_score(y_test, predictions_test)
conf_matrix_test = confusion_matrix(y_test, predictions_test)

print("\nAccuracy on Test Set:", accuracy_test)
#print("\nConfusion Matrix on Test Set:\n", conf_matrix_test)

<Figure size 432x288 with 0 Axes>

40/40 [==============================] - 0s 702us/step

Accuracy on Test Set: 0.9760574620909817


In [22]:
#------Code for Pulsar Dataset----------#
#------Standard BNN--------------#
# This code is motivated from: https://keras.io/examples/keras_recipes/bayesian_neural_networks/ 
import tensorflow as tf

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from tensorflow import keras
import tensorflow_probability as tfp
from tensorflow.keras import layers
from tensorflow.keras.optimizers import Adam
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score, confusion_matrix

# Load the data
data = pd.read_csv('pulsar_data.csv')

# Assuming 'target' is the column containing the labels
X = data.drop('target_class', axis=1)
y = data['target_class']

# Handling missing values by filling with the mean
X = X.fillna(X.mean())

# Split the data into training, testing, and validation sets
# 80% for training, 10% for validation, and 10% for testing
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.2, random_state=42)
X_validation, X_test, y_validation, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

# Scale the features using MinMaxScaler
scaler = MinMaxScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_validation_scaled = scaler.transform(X_validation)
X_test_scaled = scaler.transform(X_test)

# Define the prior weight distribution as Normal of mean=0 and stddev=1.
# Note that, in this example, the we prior distribution is not trainable,
# as we fix its parameters.
def prior(kernel_size, bias_size, dtype=None):
    n = kernel_size + bias_size
    prior_model = keras.Sequential([
        tfp.layers.DistributionLambda(
            lambda t: tfp.distributions.MultivariateNormalDiag(
                loc=tf.zeros(n), scale_diag=tf.ones(n)
            )
        )
    ])
    return prior_model

# Define variational posterior weight distribution as multivariate Gaussian.
# Note that the learnable parameters for this distribution are the means,
# variances
def posterior(kernel_size, bias_size, dtype=None):
    n = kernel_size + bias_size
    posterior_model = keras.Sequential([
        tfp.layers.VariableLayer(
            tfp.layers.MultivariateNormalTriL.params_size(n), dtype=dtype
        ),
        tfp.layers.MultivariateNormalTriL(n),
    ])
    return posterior_model

def create_bnn_model(train_size):
    model = keras.Sequential()
    model.add(layers.InputLayer(input_shape=X_train_scaled.shape[1]))

    # Create hidden layers with weight uncertainty using the DenseVariational layer.
    for units in [2,2]:  #  number of units in hidden layers
        model.add(tfp.layers.DenseVariational(
            units=units,
            make_prior_fn=prior,
            make_posterior_fn=posterior,
            kl_weight=1 / train_size,
            activation='relu',
        ))

    model.add(layers.Dense(units=1, activation='sigmoid'))
    return model

#  BNN model
bnn_model = create_bnn_model(train_size=len(X_train_scaled))

#  model with the Adam optimizer and binary cross-entropy loss
bnn_model.compile(optimizer=Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy'])

# training history
history = bnn_model.fit(X_train_scaled, y_train, validation_data=(X_validation_scaled, y_validation),
                         epochs=50, batch_size=32, verbose=0)

# Plot epoch vs accuracy and loss
plt.figure(figsize=(12, 5))

# Plot Accuracy
#plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Training Accuracy')
#plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.xlabel('Iterations')
plt.ylabel('Accuracy')
#plt.title('Epoch vs Accuracy')
plt.legend()
plt.savefig('BNN_accuracy_plot.pdf', bbox_inches='tight')
plt.close() 

# Plot Loss
#plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Training Loss')
#plt.plot(history.history['val_loss'], label='Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
#plt.title('Epoch vs Loss')
plt.legend()
plt.savefig('BNN_loss_plot.pdf', bbox_inches='tight')
plt.close()
plt.tight_layout()
plt.show()

# Make predictions on the test data
predictions_test = (bnn_model.predict(X_test_scaled) > 0.5).astype(int)

# Evaluate the model on the test set
accuracy_test = accuracy_score(y_test, predictions_test)
conf_matrix_test = confusion_matrix(y_test, predictions_test)

print("\nAccuracy on Test Set:", accuracy_test)
#print("\nConfusion Matrix on Test Set:\n", conf_matrix_test)

<Figure size 432x288 with 0 Axes>

40/40 [==============================] - 0s 898us/step

Accuracy on Test Set: 0.9784517158818835


In [9]:
#-----------------FFNN on iris-------------------#

In [29]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score, confusion_matrix

# Load the data
data = pd.read_csv('reduced_iris.csv')

# Select the first 100 rows
data = data.head(100)

# Assuming 'label' is the column containing the labels
X = data[['col1', 'col2']]  # Selecting the features
y = data['label']

# Split the data into training, testing, and validation sets
# 80% for training, 10% for validation, and 10% for testing
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.3, random_state=42)
X_validation, X_test, y_validation, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

# Scale the features using MinMaxScaler
scaler = MinMaxScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_validation_scaled = scaler.transform(X_validation)
X_test_scaled = scaler.transform(X_test)

# Initialize the neural network model with 2 hidden layers and 2 nodes in each hidden layer
model = Sequential()
model.add(Dense(units=2, input_dim=X_train_scaled.shape[1], activation='relu'))
model.add(Dense(units=2, activation='relu'))
model.add(Dense(units=1, activation='sigmoid'))

# Compile the model with the Adam optimizer and a custom learning rate
custom_optimizer = Adam(learning_rate=0.001)  # Adjust the learning rate as needed
model.compile(optimizer=custom_optimizer, loss='binary_crossentropy', metrics=['accuracy'])

# Train the model and get the training history
history = model.fit(X_train_scaled, y_train, validation_data=(X_validation_scaled, y_validation), epochs=50, batch_size=32, verbose=0)

# Plot epoch vs accuracy and loss
plt.figure(figsize=(18, 7))

# Plot Accuracy
#plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Training Accuracy')
#plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.xlabel('Iterations')
plt.ylabel('Accuracy')
#plt.title('Epoch vs Accuracy')
plt.legend()
plt.savefig('iris_FFNN_accuracy_plot.pdf', bbox_inches='tight')
plt.close()

# Plot Loss
#plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Training Loss')
#plt.plot(history.history['val_loss'], label='Validation Loss')
plt.xlabel('Iterations')
plt.ylabel('Loss')
#plt.title('Epoch vs Loss')
plt.legend()
plt.savefig('iris_FFNN_loss_plot.pdf', bbox_inches='tight')
plt.close()
plt.tight_layout()
plt.show()

# Make predictions on the test data
predictions_test = (model.predict(X_test_scaled) > 0.5).astype(int)

# Evaluate the model on the test set
accuracy_test = accuracy_score(y_test, predictions_test)
conf_matrix_test = confusion_matrix(y_test, predictions_test)

print("\nAccuracy on Test Set:", accuracy_test)
#print("\nConfusion Matrix on Test Set:\n", conf_matrix_test)

<Figure size 432x288 with 0 Axes>

1/1 [==============================] - 0s 37ms/step

Accuracy on Test Set: 0.7333333333333333


In [33]:
#------------BNN for iris data-------------------#
# This code is motivated from: https://keras.io/examples/keras_recipes/bayesian_neural_networks/
import tensorflow as tf
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from tensorflow import keras
import tensorflow_probability as tfp
from tensorflow.keras import layers
from tensorflow.keras.optimizers import Adam
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score, confusion_matrix

# Load the data
data = pd.read_csv('reduced_iris.csv')

# Select the first 100 rows
data = data.head(100)

# Assuming 'label' is the column containing the labels
X = data[['col1', 'col2']]  # Selecting the features
y = data['label']

# Split the data into training, testing, and validation sets
# 80% for training, 10% for validation, and 10% for testing
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.3, random_state=42)
X_validation, X_test, y_validation, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

# Scale the features using MinMaxScaler
scaler = MinMaxScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_validation_scaled = scaler.transform(X_validation)
X_test_scaled = scaler.transform(X_test)

# Define prior and posterior distributions (same as in the previous code)
def prior(kernel_size, bias_size, dtype=None):
    n = kernel_size + bias_size
    prior_model = keras.Sequential([
        tfp.layers.DistributionLambda(
            lambda t: tfp.distributions.MultivariateNormalDiag(
                loc=tf.zeros(n), scale_diag=tf.ones(n)
            )
        )
    ])
    return prior_model

def posterior(kernel_size, bias_size, dtype=None):
    n = kernel_size + bias_size
    posterior_model = keras.Sequential([
        tfp.layers.VariableLayer(
            tfp.layers.MultivariateNormalTriL.params_size(n), dtype=dtype
        ),
        tfp.layers.MultivariateNormalTriL(n),
    ])
    return posterior_model

# Create the BNN model
def create_bnn_model(train_size):
    model = keras.Sequential()
    model.add(layers.InputLayer(input_shape=X_train_scaled.shape[1]))

    # Create hidden layers with weight uncertainty using the DenseVariational layer
    for units in [10]:  # Adjust the number of units in hidden layers
        model.add(tfp.layers.DenseVariational(
            units=units,
            make_prior_fn=prior,
            make_posterior_fn=posterior,
            kl_weight=1 / train_size,
            activation='relu',
        ))

    model.add(layers.Dense(units=1, activation='sigmoid'))
    return model

# Create the BNN model
bnn_model = create_bnn_model(train_size=len(X_train_scaled))

# Compile the model with the Adam optimizer and binary cross-entropy loss
bnn_model.compile(optimizer=Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy'])

# Train the model and get the training history
history = bnn_model.fit(X_train_scaled, y_train, validation_data=(X_validation_scaled, y_validation),
                         epochs=50, batch_size=22, verbose=0)

# Plot epoch vs accuracy and loss
plt.figure(figsize=(18,7))

# Plot Accuracy
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Training Accuracy')
#plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.xlabel('Iterations')
plt.ylabel('Accuracy')
#plt.title('Epoch vs Accuracy')
plt.legend()
plt.savefig('iris_BNN_accuracy_plot.pdf', bbox_inches='tight')
plt.close()

# Plot Loss
#plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Training Loss')
#plt.plot(history.history['val_loss'], label='Validation Loss')
plt.xlabel('Iterations')
plt.ylabel('Loss')
#plt.title('Epoch vs Loss')
plt.legend()
plt.savefig('iris_BNN_loss_plot.pdf', bbox_inches='tight')
plt.close()
plt.tight_layout()
plt.show()

# Make predictions on the test data
predictions_test = (bnn_model.predict(X_test_scaled) > 0.5).astype(int)

# Evaluate the model on the test set
accuracy_test = accuracy_score(y_test, predictions_test)
conf_matrix_test = confusion_matrix(y_test, predictions_test)

print("\nAccuracy on Test Set:", accuracy_test)
#print("\nConfusion Matrix on Test Set:\n", conf_matrix_test)

<Figure size 432x288 with 0 Axes>

1/1 [==============================] - 0s 197ms/step

Accuracy on Test Set: 0.6666666666666666
